### Explain OverConfident Tornado Predictions. 

In [1]:
# The custom classifier 
import sys, os
sys.path.insert(0, '/home/monte.flora/python_packages/WoF_post')
sys.path.insert(0, '/home/monte.flora/python_packages/ml_workflow')
sys.path.insert(0, '/home/monte.flora/python_packages/wofs_ml_severe')

from ml_workflow import TunedEstimator 
from wofs_ml_severe import load_ml_data
from wofs_ml_severe.io.load_ml_models import load_ml_model
from wofs.post.utils import load_yaml
from wofs_ml_severe.common.util import get_target_str, fix_data
from display_names import to_display_name, to_units

import numpy as np
import pandas as pd
from os.path import join
import joblib

import shap 
import skexplain

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
retro = False
target = 'tornado_severe_0km'
file_log = 'removed_features_ls_loss'
lead_time = 'first_hour'
resample = None

In [3]:
# Load the data. 
mode = 'testing'
X_test, y_test, metadata = load_ml_data(target_col=target, 
                              lead_time=lead_time,
                              mode=mode,
                              base_path = '/work/mflora/ML_DATA/DATA/',
                            alter_init_times=True, 
                                 )
X_test = fix_data(X_test)
    
target_str = get_target_str(target)
    
print(target_str)
# Load the ML models. 
ml_config = load_yaml(
    '/home/monte.flora/python_packages/wofs_ml_severe/wofs_ml_severe/conf/ml_config_retro.yml')

parameters = {
                'target' : target,
                'time' : lead_time, 
                'model_name' : 'LogisticRegression',
                'ml_config' : ml_config,
                'file_log'  : file_log,
                'resample' : resample
            }

model_dict = load_ml_model(retro, **parameters)
model = model_dict['model']
X_train = model_dict['X']
features = X_train.columns

X_test = X_test[features]

tornado
Loading /work/mflora/ML_DATA/NEW_ML_MODELS/LogisticRegression_tornado_severe_0km_None_first_hour_realtime__removed_features_ls_loss.joblib...


FileNotFoundError: [Errno 2] No such file or directory: '/work/mflora/ML_DATA/NEW_ML_MODELS/LogisticRegression_tornado_severe_0km_None_first_hour_realtime__removed_features_ls_loss.joblib'

In [ ]:
# As stated above, the masker handles the missing features. In this case, we are using correlations 
# in the dataset to determine the feature groupings. These groups of features are remove or added into 
# sets together. 
X_train = fix_data(X_train)

# As stated above, the masker handles the missing features. In this case, we are using correlations 
# in the dataset to determine the feature groupings. These groups of features are remove or added into 
# sets together. 
shap_kws={'masker' : shap.maskers.Partition(X_train.values.astype(float), 
                                            max_samples=150, clustering="correlation"), 
           'algorithm' : 'permutation'}

In [ ]:
display_feature_names = {f: to_display_name(f) for f in features}
display_units = {f: to_units(f) for f in features}

### SHAP Values for the Highest Tornado Prediction on the Testing dataset

In [ ]:
%matplotlib inline

pred = model.predict_proba(X_test)[:,1]

inds = np.argsort(pred)[::-1]


ind = inds[10]

single_example = X_test.iloc[[ind]].astype(float)

explainer = skexplain.ExplainToolkit(('LR', model), X=single_example)
contrib_ds = explainer.local_attributions(method='shap', 
                                           shap_kws = shap_kws,
                                          )

fig, axes = explainer.plot_contributions(
    contrib = contrib_ds, 
    display_feature_names = display_feature_names, 
    display_units = display_units,
    max_display = 25
)